* Data from https://opendata.dwd.de/climate_environment/CDC/observations_global/ocean/OceanRAIN/V002/OceanRAIN-W_RV-Polarstern_DBLK_JUN2010-DEC2018_DWD-MPIM_V2_0.nc
* GeoData from https://github.com/nvkelso/natural-earth-vector/blob/master/geojson/ne_110m_admin_0_countries.geojson

In [1]:
using Dates

In [2]:
using YAXArrays,NetCDF

In [9]:
dataset=open_dataset("OceanRAIN-W_RV-Polarstern_DBLK_JUN2010-DEC2018_DWD-MPIM_V2_0.nc",driver=:netcdf)

YAXArray Dataset
Shared Axes: 
Dim{:time} Sampled{Int32} Int32[0, 1, …, 4502878, 4502879] ForwardOrdered Irregular Points
Variables: 
probability_for_rain, true_wind_speed, Ka-band_specific_oneway_attenuation, relative_humidity, theoretical_rain_rate_disdrometer, unix_epoch, Ka_band_specific_differential_phase, W2_past_weather_code, time_UT, max_gusts, S_band_reflectivity, local_time, S_band_differential_reflectivity, evaporation, Ku_band_reflectivity, C_band_differential_reflectivity, Ku_band_specific_differential_phase, bulkwater_temperature, reference_voltage, latitude, longitude, precip_flag2, relative_wind_speed_ODM470, air_pressure, Ka_band_differential_reflectivity, precip_flag, probability_for_mixed_phase, specific_humidity_at_sea_surface, theoretical_snow_rate_disdrometer, intercept_of_normalized_gamma, sensible_heat_flux_shf, particle_diameter_99th_percentile, count, dBZ, julian_date, global_radiation, mass_spectrum_standard_deviation, latent_heat_flux_lhf, convective_stratif

In [10]:
lat=dataset["latitude"].data[:]
lon=dataset["longitude"].data[:]
salinity=dataset["salinity"].data[:]
sea_surface_temperature=dataset["sea_surface_temperature"].data[:]

4400527-element Vector{Float64}:
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9
   ⋮
  -0.7
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9
 -99.9

In [11]:
lon[lon.==-999.9999].=NaN
lat[lat.==-99.9999].=NaN
salinity[salinity.==-99.99].=NaN
sea_surface_temperature[sea_surface_temperature.==-99.9].=NaN
salinity[salinity.==99.99].=NaN

443535-element view(::Vector{Float64}, [62414, 62415, 62416, 62417, 62418, 271234, 271235, 271236, 271237, 271238  …  3238359, 3238361, 3238362, 3238363, 3238364, 3238366, 3238367, 3238368, 3238369, 3238370]) with eltype Float64:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [6]:
using GLMakie,GeoMakie


In [12]:
lons = -180:180
lats = -90:90
field = [1 for l in lons, y in lats]

361×181 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 ⋮              ⋮              ⋮        ⋱     ⋮              ⋮              ⋮
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1   

In [13]:
using GeoJSON

In [14]:
worldCountries = GeoJSON.read("../ne_110m_admin_0_countries.geojson")

FeatureCollection with 177 Features

In [31]:
fig=Figure(backgroundcolor=:gray)
Label(fig[1,1:2], text="Polarstern route June 2010 - December 2018\n with sea surface temperature",fontsize=30)
ga1 = GeoAxis(fig[2, 1]; dest = "+proj=ortho +lon_0=-100 +lat_0=90", coastlines = false,latlims=(0,90),aspect=1,backgroundcolor=:gray,width=400,height=400)
hm1=surface!(ga1,lons,lats,field,colormap=:blues)
translate!(hm1, 0, 0, -100)
scatter!(ga1,lon,lat,markersize=0.1,color=sea_surface_temperature,colormap=cgrad(:managua,rev=true))
hm2 = poly!(
    ga1, worldCountries;
    color= :gray,
    strokecolor = :black,
    strokewidth = 0.25
)

hidespines!(ga1)
hidedecorations!(ga1,grid=false)
ga1 = GeoAxis(fig[2, 2]; dest = "+proj=ortho +lon_0=80 +lat_0=-90", coastlines = false,latlims=(-85,0),aspect=1,backgroundcolor=:gray,width=400,height=400)
hm2 = poly!(
    ga1, worldCountries;
    color= :gray,
    strokecolor = :black,
    strokewidth = 0.25
)
hm1=surface!(ga1,lons,lats,field,colormap=:blues)
translate!(hm1, 0, 0, -100)
p=scatter!(ga1,lon,lat,markersize=0.1,color=sea_surface_temperature,colormap=cgrad(:managua,rev=true))

Colorbar(fig[3,1:2],p,size=50, label="Sea surface temperature in °C",vertical=false,labelsize=20)
hidespines!(ga1)
hidedecorations!(ga1,grid=false)
Label(fig[4,1:2],text="Data: © Deutscher Wetterdienst\nMade with Natural Earth",justification=:left)
Makie.resize_to_layout!(fig)
fig

In [32]:
save("polarstern_seasurfacetemperature.png",fig,pt_per_pixel=2)